# Test The Model Deployed from Our Pipeline

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Retrieve Endpoint Name  

In [2]:
%store -r step_functions_pipeline_endpoint_name

In [3]:
print(step_functions_pipeline_endpoint_name)

training-pipeline-2020-08-22-22-27-38


# Wait For The Endpoint To Be Deployed

In [4]:
client = boto3.client('sagemaker')
waiter = client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=step_functions_pipeline_endpoint_name)

# Invoke the Endpoint

In [5]:
import json
from sagemaker.tensorflow.serving import Predictor

predictor = Predictor(endpoint_name=step_functions_pipeline_endpoint_name,
                      sagemaker_session=sess,
                      content_type='application/json',
                      model_name='saved_model',
                      model_version=0)

In [6]:
reviews = ["This is great!"]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

[Predicted Star Rating: 1] This is great!


# Delete Endpoints
To save money, we should delete the endpoint.

In [7]:
sm.delete_endpoint(
    EndpointName=step_functions_pipeline_endpoint_name
)

{'ResponseMetadata': {'RequestId': '41de544d-51eb-4747-af41-caca01f30f55',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '41de544d-51eb-4747-af41-caca01f30f55',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 22 Aug 2020 22:47:56 GMT'},
  'RetryAttempts': 0}}

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>